In [1]:
!pip install transformers -q
!pip install sentencepiece -q

     |████████████████████████████████| 3.1 MB 14.2 MB/s 
     |████████████████████████████████| 895 kB 74.1 MB/s 
     |████████████████████████████████| 59 kB 9.3 MB/s 
     |████████████████████████████████| 3.3 MB 81.4 MB/s 
     |████████████████████████████████| 596 kB 86.2 MB/s 
     |████████████████████████████████| 1.2 MB 11.8 MB/s 


In [2]:
# Mount Google drive to upload datasets
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The path to the data on my drive
D = '/content/drive/My Drive/W266_Project_Data/pmi_data'

In [4]:
import pandas as pd
import csv
from transformers import AutoTokenizer, AutoModel
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [5]:
# Load the gold-standard target translation datasets
hi_target = pd.read_csv(D+"/predicted_text/MBart/dev.hi", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])
ml_target = pd.read_csv(D+"/predicted_text/MBart/dev.ml", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])
ta_target = pd.read_csv(D+"/predicted_text/MBart/dev.ta", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_1"])

In [6]:
hi_target.head()

,sentence_1
0,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...
1,इस समारोह को आज बीजापुर में आयोजित करने के महत...
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [7]:
# Load the Bart translated files into their respective language datasets
hi_bart = pd.read_csv(D+"/predicted_text/MBart/dev_bart_translated_hi", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_2"])
ml_bart = pd.read_csv(D+"/predicted_text/MBart/dev_bart_translated_ml", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_2"])
ta_bart = pd.read_csv(D+"/predicted_text/MBart/dev_bart_translated_ta", sep = "\t", quoting=csv.QUOTE_NONE, names=["sentence_2"])

In [8]:
hi_bart.head()

,sentence_2
0,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...
1,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...
2,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [9]:
# Create Hindi Bart test set
hi_bart_test = pd.concat([hi_target['sentence_1'], hi_bart['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
hi_bart_test.insert(0, 'language_task', 'translate English to Hindi')
hi_bart_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [10]:
# Create Malayalam Bart test set
ml_bart_test = pd.concat([ml_target['sentence_1'], ml_bart['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
ml_bart_test.insert(0, 'language_task', 'translate English to Malayalam')

In [11]:
# View first few rows of Malayalam Bart test set
ml_bart_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Malayalam,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില് അഭിലാഷങ്ങള് സൃ...
1,translate English to Malayalam,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,രാജ്യത്തിന്റെ വികസയാത്രയില് പിന്നോക്കം വിട്ടുപ...
2,translate English to Malayalam,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,2022 ഓടെ ഈ ലക്ഷ്യം പൂര് ത്തിയാക്കും.
3,translate English to Malayalam,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,സമയബന്ധമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടെയും ഗവണ്മെന...
4,translate English to Malayalam,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ജന് ധന് അക്കൗണ്ട് തുറക്കാന്, ദരിദ്രര് ക്ക് ഇന്..."


In [12]:
# Load the first few rows of the Tamil Bart test set
ta_bart_test = pd.concat([ta_target['sentence_1'], ta_bart['sentence_2']], axis=1, keys=['sentence_1', 'sentence_2'])
ta_bart_test.insert(0, 'language_task', 'translate English to Tamil')

In [13]:
# View the first few rows of the Tamil translation
ta_bart_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Tamil,கோடிக்கணக்கான மக்களின் மனங்களிலும் மற்றும் இதய...,கோடிக்கணக்கான மக்களின் மனங்களிலும் இதயங்களிலும...
1,translate English to Tamil,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...,பீஜப்பூர் பகுதியில் இந்த நிகழ்ச்சியை நடத்துவதன...
2,translate English to Tamil,இந்தப் பணியை 2022-ம் ஆண்டுக்குள் நிறைவேற்றுவதே...,2022 ஆம் ஆண்டுக்குள் இந்த பணியை நிறைவேற்றுவதை ...
3,translate English to Tamil,மத்திய அரசு பல்வேறு திட்டங்கைளை தெளிவான இலக்கு...,மத்திய அரசு பல்வேறு திட்டங்களை தெளிவான இலக்குக...
4,translate English to Tamil,"இது தொடர்பாக ஜன் தன் கணக்குகள், ஏழை மக்களுக்கா...","ஜன் தன் கணக்குகள், ஏழை மக்களுக்கான காப்பீடு, உ..."


In [14]:
# Re-define the Siamese Neural Network since the class wouldn't load
class SiameseNet(torch.nn.Module):
    def __init__(self):
      super(SiameseNet, self).__init__()

      # Set up a fully-connected layer to apply to each incoming sentence vector
      # Use a dense layer (nn.Linear) to reduce the dimensionality of the input embeddings
      # use a relu activation function
      # Randomly drop out
      # Set 1 as the dimensionality of the final output since we want to predict the logits corresponding to a binary label
      # Initialize the feed forward layer
      
      self.relu = torch.nn.ReLU(inplace=True)
      self.batch2 = torch.nn.InstanceNorm1d(2304)
      self.ffn = torch.nn.Linear(2304, 128)
      self.batch3 = torch.nn.InstanceNorm1d(128)
      self.layer_out = torch.nn.Linear(128,1)

      # Add drop out
      self.drop = torch.nn.Dropout(0.2)

    # left and right represent columns in our data
    def forward(self, left, right):

      # Get the sentence embeddings 
      u = left
      v = right

      # Get the absolute value of the difference of the vectors
      uv = torch.sub(u, v)
      uv_abs = torch.abs(uv)

      # concatenate u, v, and the absolute value of the element-wise difference between the embeddings
      x = torch.cat([u, v, uv_abs], dim=-1)

      # Normalize the concatenated batch
      x = self.batch2(x)

      # Use the relu activation function
      x = self.relu(x)

      # Pass the concatenated layers through a dense layer to reduce the dimensionality from 2304 to 128
      x = self.ffn(x)
      # Normalize the concatenated layers
      x = self.batch3(x)
      # Pass through relu activation
      x = self.relu(x)
      # Implement drop out to avoid overfitting
      x = self.drop(x)
      # Pass the output through a final dense layer
      output = self.layer_out(x)
      # Reshape the output to have the same dimension as the target labels
      output = output[:,-1,:]

      # Concatenate the 
      return output

In [15]:
# Set up the device to run on GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [16]:
# Create an instance of the tokenizer and model using Indic Bert
tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert', keep_accents=True)
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

Downloading:   0%|          | 0.00/507 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.38M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/129M [00:00<?, ?B/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['predictions.dense.weight', 'predictions.LayerNorm.bias', 'sop_classifier.classifier.bias', 'predictions.decoder.weight', 'sop_classifier.classifier.weight', 'predictions.dense.bias', 'predictions.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
# Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def sentence_embeddings(device, tokenizer, model, sentences):
  """Create contextualized sentence embeddings
  
  Parameters:
  tokenizer: transformers tokenizer for creating word tokens
  model:
  sentences: sentences to generate sentence embeddings for
  """

  # Tokenize sentences and return PyTorch tensors
  encoded_input = tokenizer(sentences, padding=True, truncation=True, 
                            max_length=128, return_tensors='pt')
  encoded_input = encoded_input.to(device)
  
  # Pass the tokenized input to the model 
  with torch.no_grad():
    model.to(device)
    model_output = model(**encoded_input) 
  
  # Perform mean pooling to get total sentence embeddings
  sentence_embeddings = mean_pooling(model_output, 
                                     encoded_input['attention_mask'])
  
  return sentence_embeddings

In [20]:
# Load the Siamese Neural Net from the pre-trained weights
net = SiameseNet().to(device)
# Load the weights and biases from the last round of training
net.load_state_dict(torch.load("/content/drive/My Drive/W266_Project_Data/models/siamese_model/model_v3.pt"), strict=False)

_IncompatibleKeys(missing_keys=[], unexpected_keys=['fc.0.weight', 'fc.0.bias'])

In [21]:
# Create a dataset class, similarly to the one used for training the Siamese Neural Network dataset - the difference is that it won't create labels
class TranslationEvalDataset(Dataset): 
  def __init__(self,training_df, device, tokenizer, model):
    self.training_df=training_df
    self.sent1 = self.training_df['sentence_1'].tolist()
    self.sent2 = self.training_df['sentence_2'].tolist()
    self.language_task = self.training_df['language_task'].tolist()
    self.device = device
    self.tokenizer = tokenizer
    self.model = model

  def __getitem__(self, index):
    sent1 = self.sent1[index]
    sent2 = self.sent2[index]
    task = self.language_task[index]

    # Get the sentence embeddings for sentences 1 and 2 for each row
    sent1_embeddings = sentence_embeddings(self.device, self.tokenizer, self.model, sent1)
    sent2_embeddings = sentence_embeddings(self.device, self.tokenizer, self.model, sent2)
    return sent1_embeddings, sent2_embeddings, task

  def __len__(self):
    return len(self.training_df)

In [22]:
# Concatenate all of the bart test sets into one test set
final_test = pd.concat([hi_bart_test, ml_bart_test, ta_bart_test]).reset_index()
final_test.drop(columns={'index'}, inplace=True)

In [23]:
final_test.head()

,language_task,sentence_1,sentence_2
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो..."


In [24]:
final_test[1000:2000]

,language_task,sentence_1,sentence_2
1000,translate English to Malayalam,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില്‍ അഭിലാഷങ്ങള്‍ ...,കോടിക്കണക്കിന് ജനങ്ങളുടെ മനസില് അഭിലാഷങ്ങള് സൃ...
1001,translate English to Malayalam,രാജ്യത്തിന്റെ വികസയാത്രയില്‍ പിന്നോക്കം വിട്ടു...,രാജ്യത്തിന്റെ വികസയാത്രയില് പിന്നോക്കം വിട്ടുപ...
1002,translate English to Malayalam,2022 ഓടെ ഈ ലക്ഷ്യം പൂര്‍ത്തീകരിക്കുകയാണ് ലക്ഷ്...,2022 ഓടെ ഈ ലക്ഷ്യം പൂര് ത്തിയാക്കും.
1003,translate English to Malayalam,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,സമയബന്ധമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടെയും ഗവണ്മെന...
1004,translate English to Malayalam,ജന്‍ ധന്‍ അക്കൗണ്ടുകള്‍ തുറക്കുന്നതിലും ദരിദ്ര...,"ജന് ധന് അക്കൗണ്ട് തുറക്കാന്, ദരിദ്രര് ക്ക് ഇന്..."
...,...,...,...
1995,translate English to Malayalam,മ്യാന്‍മാറിലെ യൂണിയന്‍ തിരഞ്ഞെടുപ്പ് കമ്മിഷന്‍ 7.,യൂണിയന് ရွေးကောက်ပွဲ് കമ്മിഷന് 7.
1996,translate English to Malayalam,സ്വാതന്ത്ര്യാനന്തരം പല നേട്ടങ്ങളും ഉണ്ടാക്കാന്...,സ്വതന്ത്രനെന്നതരം പല നേട്ടങ്ങള് ഉണ്ടാക്കാന് കഴ...
1997,translate English to Malayalam,"ജന്‍ ഭാഗീദാരി, വിവരം പങ്കിടല്‍, വിഭവങ്ങള്‍ പരമ...","ജാന് ഭാഗീദാരി, വിവരങ്ങള് പങ്കുവയ്ക്കാന്, വിഭവങ..."
1998,translate English to Malayalam,"ഗ്രാമ സ്വരാജ് അഭിയാന്‍, ആയുഷ്മാന്‍ ഭാരത് തുടങ്...","ഗ്രാമ സ്വ Raj അഭിയാന്, ആയുഷ്മാന് ഇന്ത്യ് തുടങ്..."


In [25]:
final_test.tail()

,language_task,sentence_1,sentence_2
2995,translate English to Tamil,6. மியான்மர் மத்திய தேர்தல் ஆணையம்,6. மியான்மர் மத்திய தேர்தல் ஆணையம்
2996,translate English to Tamil,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...,“சுதந்திரம் பெற்ற பிறகு ஏராளமானவை நிறைவேற்றப்ப...
2997,translate English to Tamil,"பொது மக்களின் பங்களிப்பு, தகவல் ஓட்டத்தின் முற...","பொது மக்களின் பங்களிப்பு, தகவல் ஓட்ட முறை, வளங..."
2998,translate English to Tamil,"ஊரக சுயாட்சி திட்டம், வளமான பாரதம் போன்ற சமீபத...","ஊரக சுயாட்சி திட்டம்,வளமான இந்தியா போன்ற சமீபத..."
2999,translate English to Tamil,இந்தியாவின் 130 லட்சம் மக்கள் சார்பில் உங்கள் ...,130 லட்சம் இந்தியர்களுக்காக புதுதில்லிக்கு உங்...


In [26]:
# Create an instance of the test set with all test translations
bart_test = TranslationEvalDataset(final_test, device, tokenizer, model)

In [27]:
# Create an instance of the test data loader
bart_test_loader = DataLoader(bart_test, batch_size=16)

In [28]:
# Track the language_tasks and sentences  being evaluated
y_language_task = final_test.language_task.tolist()
y_sent1 = final_test.sentence_1.tolist()
y_sent2 = final_test.sentence_2.tolist()
probability_of_paraphrase = []

with torch.no_grad():

  # Place the model into evaluation mode
    net.eval()

    # Loop over the test set
    for i, data in enumerate(bart_test_loader, 0):

      # Extract the test set components
      sent1_test, sent2_test, task_test = data

      # Send the test data to GPU
      sent1_test, sent2_test = sent1_test.to(device), sent2_test.to(device)

      # Get the test set predictions
      test_pred = net(sent1_test, sent2_test)
            
      # Get the predicted test labels
      y_test_pred = torch.sigmoid(test_pred).float()

      # Add the test labels to their respective list
      probability_of_paraphrase.extend(y_test_pred)

In [29]:
# Create a dataframe of the tasks, sentences, and probabilities of the second sentence (the translated one) being a paraphrase
y_probs = [x.item() for x in probability_of_paraphrase]
bart_performance = pd.DataFrame(list(zip(y_language_task, y_sent1, y_sent2, y_probs)),columns =['language_task', 'target_sentence', 'bart_translation', 'probability_of_paraphrase'])

In [30]:
# Check the first few rows
bart_performance.head()

,language_task,target_sentence,bart_translation,probability_of_paraphrase
0,translate English to Hindi,प्रधानमंत्री ने कहा कि बाबा साहेब अम्बेडकर की ...,प्रधानमंत्री ने कहा कि बाबासाहब अम्बेडकर की कर...,0.984450
1,translate English to Hindi,इस समारोह को आज बीजापुर में आयोजित करने के महत...,व्याख्या करते हुए इस समारोह को आज बीजापुर में ...,0.982406
2,translate English to Hindi,उन्होंने कहा कि इस कार्य को 2022 तक पूरा कर ले...,उन्होंने कहा कि इस कार्य को 2022 तक पूरा करने ...,0.983475
3,translate English to Hindi,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,प्रधानमंत्री ने कहा कि सरकार स्पष्ट लक्ष्यों औ...,0.985448
4,translate English to Hindi,"उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...","उन्होंने इस संदर्भ में जन धन खाता खोलने, गरीबो...",0.985879


In [31]:
# Sort the dataframe in order of lowest predicted probability and see where BART returns the lowest scores
# The lowest quality translations, as measured by paraphrase score, are those of Malayalam and Tamil
bart_performance.sort_values("probability_of_paraphrase")

,language_task,target_sentence,bart_translation,probability_of_paraphrase
1221,translate English to Malayalam,ഡിജിറ്റല്‍ വിപ്ലവത്തോടെ രാജ്യത്തെ ശക്തിപ്പെടുത...,പ്രസിഡന് റ് പ്രസിഡന് റ് പ്രിന് റ് പ്രസിഡന് റ് ...,0.028023
2502,translate English to Tamil,ரமலான் புனித மாதம் என்பது நபிகள் நாயகத்தின் போ...,நேபாளத்தில் உள்ள நேபாளத்தில் உள்ள நேபாளத்தில் ...,0.028492
2103,translate English to Tamil,ஸ்ருதிகளில் இருந்து ஸ்ம்ரிதி வரை இந்த பாரம்பரி...,இத்தகைய விலங்குகள் விலங்குகள் எனப்படும் இந்த வ...,0.029368
1824,translate English to Malayalam,ശക്തവും പുരോഗമനപരവുമായ ഒരു കിഴക്കൻ ഇന്ത്യ കരുത...,"പശ്ചിമജര് മന് പശ്ചിമജര് മന് പശ്ചിമജര് മന്, പശ്...",0.029480
2359,translate English to Tamil,"சட்டத்தில் இப்போதுள்ள வார்த்தைகள் காரணமாக, அடு...","தற்போதுள்ள சட்டத்தின்படி, அடுத்த தகுதியானவர் த...",0.030883
...,...,...,...,...
724,translate English to Hindi,प्रधानमंत्री ने बांध पर सरदार पटेल के दृष्टिको...,प्रधानमंत्री ने बांध पर सरदार पटेल के दृष्टिको...,0.987508
839,translate English to Hindi,इस संदर्भ में पुराने नियम कायदों को दरकिनार कर...,इस संदर्भ में पुराने नियम कायदों को दरकिनार कर...,0.987749
213,translate English to Hindi,मालदीव ने भी 2020-2021 के लिए अस्‍थायी सीट के ...,मालदीव ने भी 2020-2021 के लिए अस् थायी सीट के ...,0.987762
679,translate English to Hindi,प्रधानमंत्री ने उद्मियों को अपने कर्मचारियों क...,प्रधानमंत्री ने उद्मियों को अपने कर्मचारियों क...,0.987815


In [32]:
# Check the number of rows with a predicted probability <0.5 (meaning that they would be assigned a label of 0 - not a paraphrase)
len(bart_performance[bart_performance.probability_of_paraphrase < 0.5])

490

In [33]:
# Check the number of translations of each language where the translation is not counted as a paraphrase
hi_errors = len(bart_performance[(bart_performance.probability_of_paraphrase < 0.5) & (bart_performance.language_task == 'translate English to Hindi')])
ml_errors = len(bart_performance[(bart_performance.probability_of_paraphrase < 0.5) & (bart_performance.language_task == 'translate English to Malayalam')])
ta_errors = len(bart_performance[(bart_performance.probability_of_paraphrase < 0.5) & (bart_performance.language_task == 'translate English to Tamil')])
print(hi_errors)
print(ml_errors)
print(ta_errors)

2
298
190


In [34]:
# The comparatively high error rate for Malayalam and Tamil reflect their comparatively lower Sacre Bleu scores
hi_error_rate = (hi_errors/1000)
ml_error_rate = (ml_errors/1000)
ta_error_rate = (ta_errors/1000)

In [35]:
# Hindi performs very well
print("{0:.2%}".format(hi_error_rate))

0.20%


In [36]:
# Malayalam error rate is nearly 30%
print("{0:.2%}".format(ml_error_rate))

29.80%


In [37]:
# The Tamil error rate is high, but not as high as Malayalam
print("{0:.2%}".format(ta_error_rate))

19.00%


In [38]:
# Get the sentence embeddings for a pair of "dissimilar sentences"
sent1_embed = sentence_embeddings(device, tokenizer, model, bart_performance['target_sentence'][2641])
sent2_embed = sentence_embeddings(device, tokenizer, model, bart_performance['bart_translation'][2641])

In [39]:
# Reshape to calculate the cosine similarity
sent1_embed.squeeze(0).shape

torch.Size([768])

In [40]:
# Find the cosine similarity score between these pair of "dissimilar sentences"
cosine_similarity_value = F.cosine_similarity(sent1_embed.squeeze(0), sent2_embed.squeeze(0), dim=0)

print(cosine_similarity_value)

tensor(0.9669, device='cuda:0')


In [41]:
# Try for another pair of sentences, this time, for those with a high similarity score
sent3_embed = sentence_embeddings(device, tokenizer, model, bart_performance['target_sentence'][1])
sent3_embed = sentence_embeddings(device, tokenizer, model, bart_performance['bart_translation'][1])

In [42]:
# The sentences are the same
F.cosine_similarity(sent3_embed.squeeze(0), sent3_embed.squeeze(0), dim=0)

tensor(1., device='cuda:0')

In [43]:
# Get the sentence embeddings and cosine similarity scores between each pair of sentences in the dataset
# This is to check if low scoring sentences have exact similarity scores or if they are of low quality
def check_cosine_similarity(df, device, tokenizer, model):

  # Create an empty list to check cosine similarity scores
  cosine_scores = []

  for index, rows in df.iterrows():
    # Get the sentence embeddings of each pair of target sentences
    sent1_embed = sentence_embeddings(device, tokenizer, model, rows['target_sentence'])
    sent2_embed = sentence_embeddings(device, tokenizer, model, rows['bart_translation'])

    # Reshape the embeddings to be of a single dimension 
    # Get the cosine similarity between the two pairs of sentence embeddings
    cosine_similarity_value = F.cosine_similarity(sent1_embed.squeeze(0), sent2_embed.squeeze(0), dim=0)

    # Add the cosine similarity score to the list of scores
    cosine_scores.append(cosine_similarity_value.item())

  return cosine_scores

In [44]:
translation_scores = check_cosine_similarity(bart_performance, device, tokenizer, model)

In [45]:
# Add the translation similarity scores to the indictrans performance dataframe
bart_performance['cosine_similarity_scores'] = translation_scores

In [46]:
# Sentneces with low cosine similarity also have low probability of paraphrase
bart_performance.sort_values('cosine_similarity_scores')

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
2961,translate English to Tamil,சுவீடன் அரசர் மாட்சிமை பொருந்திய கார்ல் பதினாற...,பத்ம விபூரண விபூரண விபூரண விபூரண விபூரண விபூரண...,0.040226,0.286790
2040,translate English to Tamil,"நம்மை எதிரிகளிடமிருந்தும், இயற்கைச் சீற்றம் மற...","இத்தகைய விலங்குகள், விலங்குகள், விலங்குகள், வி...",0.046731,0.293845
1292,translate English to Malayalam,സഹകരണവും ആശയവിനിമയവും ആവശ്യമുള്ള എല്ലാ വിഷയങ്ങ...,കമ്മ്യൂണിസ്റ്റ് കമ്മ്യൂണിസ്റ്റ് കമ്മ്യൂണിസ്റ്റ...,0.033301,0.361691
2930,translate English to Tamil,புராதனமானது என்பதாலேயே ஒரு கருத்தை மறுதலிப்பது...,"குறிப்பாக, குறிப்பாக, குறிப்பாக, குறிப்பாக, கு...",0.038643,0.371510
2390,translate English to Tamil,சிவாஜி மகராஜ் அனைத்திற்கும் மேலாக தனது மக்களின...,பத்மசிறீவியின் மகனான பத்மசிறீவின் மகனான பத்மசி...,0.036380,0.384583
...,...,...,...,...,...
574,translate English to Hindi,"वेतनभोगी, मध्यम वर्ग को दी गई टैक्स राहत के लि...","वेतनभोगी, मध्यम वर्ग को दी गई टैक्स राहत के लि...",0.986380,1.000000
575,translate English to Hindi,ये बजट हर भारतीय की आशाओं-आकांक्षाओ पर खरा उतर...,ये बजट हर भारतीय की आशाओं-आकांक्षाओ पर खरा उतर...,0.985926,1.000000
576,translate English to Hindi,"प्रधानमंत्री ने कहा, ‘मैं नेताजी सुभाष चन्द्र ...","प्रधानमंत्री ने कहा, ‘मैं नेताजी सुभाष चन्द्र ...",0.986413,1.000000
2262,translate English to Tamil,ஜப்பானுடனான நமது உறவு சிறப்பான உத்தி நிறைந்த ச...,ஜப்பானுடனான நமது உறவு சிறப்பான உத்தி நிறைந்த ச...,0.982656,1.000000


In [47]:
# High similarity scores have a high probability of paraphrase
bart_performance.sort_values('cosine_similarity_scores', ascending=False)

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
338,translate English to Hindi,प्रधानमंत्री इंदौर में आयोजित होने वाले शहरी व...,प्रधानमंत्री इंदौर में आयोजित होने वाले शहरी व...,0.986434,1.000000
410,translate English to Hindi,"भारत का तेजी से विकास पूरे क्षेत्र, विशेषकर श्...","भारत का तेजी से विकास पूरे क्षेत्र, विशेषकर श्...",0.985529,1.000000
407,translate English to Hindi,"हमारी शक्ति हमारे ज्ञान, क्षमता और समृद्धि को ...","हमारी शक्ति हमारे ज्ञान, क्षमता और समृद्धि को ...",0.984364,1.000000
2853,translate English to Tamil,பொதுக்கூட்டத்திலும் அவர் உரையாற்றுவார்.,பொதுக்கூட்டத்திலும் அவர் உரையாற்றுவார்.,0.983932,1.000000
402,translate English to Hindi,मेरे तमिल भाई-बहन भी काशी विश्वनाथ की भूमि वार...,मेरे तमिल भाई-बहन भी काशी विश्वनाथ की भूमि वार...,0.985802,1.000000
...,...,...,...,...,...
2390,translate English to Tamil,சிவாஜி மகராஜ் அனைத்திற்கும் மேலாக தனது மக்களின...,பத்மசிறீவியின் மகனான பத்மசிறீவின் மகனான பத்மசி...,0.036380,0.384583
2930,translate English to Tamil,புராதனமானது என்பதாலேயே ஒரு கருத்தை மறுதலிப்பது...,"குறிப்பாக, குறிப்பாக, குறிப்பாக, குறிப்பாக, கு...",0.038643,0.371510
1292,translate English to Malayalam,സഹകരണവും ആശയവിനിമയവും ആവശ്യമുള്ള എല്ലാ വിഷയങ്ങ...,കമ്മ്യൂണിസ്റ്റ് കമ്മ്യൂണിസ്റ്റ് കമ്മ്യൂണിസ്റ്റ...,0.033301,0.361691
2040,translate English to Tamil,"நம்மை எதிரிகளிடமிருந்தும், இயற்கைச் சீற்றம் மற...","இத்தகைய விலங்குகள், விலங்குகள், விலங்குகள், வி...",0.046731,0.293845


In [48]:
# Find the range in cosine similarity scores for sentences not considered paraphrases of each other
np = bart_performance[bart_performance.probability_of_paraphrase < 0.5]

In [49]:
# Find the min and max of the cosine similarity scores
print(min(np.cosine_similarity_scores))
# There are some sentences labeled as not paraphrases of each other that have a high degree of cosine similarity
print(max(np.cosine_similarity_scores))

0.28679001331329346
0.9839580655097961


In [50]:
# Do the same for sentences considered paraphrases of each other
p = bart_performance[bart_performance.probability_of_paraphrase >= 0.5]
print(min(p.cosine_similarity_scores))
print(max(p.cosine_similarity_scores))

0.6375412940979004
1.0


In [51]:
# Look at the pair of sentences with the highest degree of cosine similarity
np[np['cosine_similarity_scores'] > 0.5]

,language_task,target_sentence,bart_translation,probability_of_paraphrase,cosine_similarity_scores
445,translate English to Hindi,"वे अत्यधिक energetic, innovative और focussedहो...","वे अत्यधिक ऊर्जावान, नवान्वेषी और केंद्रित हैं।",0.053610,0.755406
545,translate English to Hindi,इसमें Ease of doing business के साथ ही Ease of...,इसमें व्यवसाय करने की आसानता के साथ ही जीवन की...,0.331935,0.845641
1003,translate English to Malayalam,സമയബന്ധിതമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടുംകൂടി ഗവണ...,സമയബന്ധമായും കൃത്യമായ ലക്ഷ്യങ്ങളോടെയും ഗവണ്മെന...,0.160481,0.913196
1012,translate English to Malayalam,സഹകരിക്കാവുന്ന മേഖലകള്‍ സംബന്ധിച്ച കാര്യങ്ങള്‍...,കമ്മ്യൂണിറ്റിന് റെ കമ്മ്യൂണിറ്റിന് റെ രൂപീകരണം.,0.093672,0.746207
1014,translate English to Malayalam,അദ്ദേഹത്തെ കാണുക എന്നത് ആഹ്ലാദകരമാണ്.,അദ്ദേഹത്തെ കാണാൻ സന്തോഷകരമാണ്.,0.360399,0.862832
...,...,...,...,...,...
2947,translate English to Tamil,"தொழில்முனைவோர் ஊக்குவிப்பு, எளிதாக தொழில் செய்...","தொழில்முனைவோரை ஊக்குவிப்பது, எளிதில் தொழில் தொ...",0.107917,0.854939
2962,translate English to Tamil,"தூயத் தொழில்நுட்பங்கள், சுற்றுச்சூழல் தீர்வுகள...","தூய்மையான தொழில்நுட்பங்கள், சுற்றுச்சூழல் தீர்...",0.166597,0.878478
2967,translate English to Tamil,"இந்நிலையில், கல்வி நிறுவனங்களுக்கும், ஆராய்ச்ச...","இந்த வகையில், கல்வி நிறுவனங்கள், ஆராய்ச்சி மேம...",0.445078,0.980516
2980,translate English to Tamil,சமீபத்தில் துவங்கப்பட்ட அந்த்யோதயா விரைவு ரயில...,அண்மையில் தொடங்கப்பட்ட அந்தோதயா விரைவு ரயில் வ...,0.184898,0.963187


In [52]:
# Get the target sentence of the pair with maximum cosine similarity (that are considered not a paraphrase)
np.loc[np.cosine_similarity_scores == max(np.cosine_similarity_scores), 'target_sentence'].iloc[0]

'நிகழ்ச்சியில் பேசிய பிரதமர், இந்த மையம் இந்தியாவின் பொருளாதார வளர்ச்சி, உயர்ந்த கலாச்சாரம் மற்றும் பாரம்பரியம், சுற்றுச்சூழலை பாதுகாப்பது தொடர்பான நமது மனநிலை போன்றவற்றை பிரதிபலிப்பதாக அமையும் என்று கூறினார்.'

In [54]:
# Get the translation sentence of the pair with maximum cosine similarity (that are considered not a paraphrase)
np.loc[np.cosine_similarity_scores == max(np.cosine_similarity_scores), 'bart_translation'].iloc[0]

'நிகழ்ச்சியில் பேசிய பிரதமர், இந்த மையம் இந்தியாவின் பொருளாதார வளர்ச்சி, உயர்ந்த கலாச்சாரம் மற்றும் பாரம்பரியம், சுற்றுச்சூழல் பாதுகாப்பு தொடர்பான நமது மனநிலை ஆகியவற்றை பிரதிபலிக்கிறது என்றார்.'

In [55]:
# The English translation of the target sentence with the maximum cosine similarity score - using Google translate to translate to English
english_np_target = 'Speaking on the occasion, the Prime Minister said that the Center would reflect on India\'s economic development, high culture and heritage, and our attitude towards environmental protection.'
english_np_translation = 'Speaking on the occasion, the Prime Minister said the center reflects India\'s economic development, high culture and heritage, and our attitude towards environmental protection.'
# As we can see in the examples below, the verb tenses are different - would reflect versus currently reflect
print(english_np_target)
print(english_np_translation)

Speaking on the occasion, the Prime Minister said that the Center would reflect on India's economic development, high culture and heritage, and our attitude towards environmental protection.
Speaking on the occasion, the Prime Minister said the center reflects India's economic development, high culture and heritage, and our attitude towards environmental protection.


In [56]:
# Let's look at the sentence pair with the lowest cosine similarity in the paraphrase dataset
print(p.loc[p.cosine_similarity_scores == min(p.cosine_similarity_scores), 'target_sentence'].iloc[0])
print(p.loc[p.cosine_similarity_scores == min(p.cosine_similarity_scores), 'bart_translation'].iloc[0])

ആയുരാരോഗ്യ സൗഖ്യത്തിനുള്ള ഏറ്റവും വലിയ ജനകീയ മുന്നേറ്റങ്ങളിലൊന്നായി യോഗാദിനം മാറിയിരിക്കുകയാണെന്നും അദ്ദേഹം കൂട്ടിച്ചേര്‍ത്തു.
അദ്ദേഹം കൂട്ടിച്ചേര് ത്തു.


In [57]:
# Print the English translations of these sentences - obtained using Google translate to back-translate
# These sentences reflect an error of the model - the Bart translation is grammatically incorrect due to truncation and the sentence has no meaning
english_p_target = 'He added that Yoga Day has become one of the biggest popular initiatives for longevity.'
english_p_translation = 'He added.'

In [60]:
# Write the bart performance file to a csv 
bart_performance.to_csv(D+"/predicted_text/siamese_evaluations_of_translations/bart_siamese_paraphrase_eval.csv")